# Timer

In [3]:
from time import sleep
from datetime import datetime as dt

start_time = "13.01.2023, 17:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI', 'LinearReg']
indicator_list = pattern
indicator_list_higher = ['LinearReg']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [2, 3, 4, 5, 6, 8], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 6


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:23<00:00,  3.86s/it]


# Check local statistics

In [2]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_rank', ascending=False).head(20)
stat

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,STOCH_RSI_LinearReg,14,35,9,7,3,25,2,0,89,2.082854,-0.743750,96.374021,-66.193750,1.3885,1.7580,2.0169,2.2102,2.9588,2.7134,2.5099,2.5518,2.4089,2.3219,2.2839,2.2066,2.0612,1.9904,1.9476,1.9393,1.8711,1.8374,1.8426,1.8320,1.8520,1.8694,1.8726,1.7441,-0.27,-0.36,-0.55,-0.71,-0.96,-0.89,-0.79,-0.83,-0.75,-0.70,-0.68,-0.66,-0.67,-0.53,-0.48,-0.59,-0.74,-0.80,-0.94,-1.02,-1.01,-1.10,-0.95,-0.87
4,STOCH_RSI_LinearReg,14,35,9,7,3,25,6,0,31,4.067146,-1.475833,95.081521,-45.750833,1.8029,2.5270,3.0042,3.3543,5.4796,4.6616,4.6309,4.8616,4.8506,4.8110,4.6867,4.6581,4.5589,4.2715,4.1212,4.0553,3.9832,3.8882,3.9264,3.8738,3.8778,3.8958,3.9020,3.9289,-0.24,-0.44,-0.69,-0.98,-1.47,-1.21,-1.75,-2.10,-1.96,-1.67,-1.59,-1.40,-1.25,-0.95,-1.25,-1.29,-1.04,-1.38,-1.86,-1.51,-1.87,-2.08,-2.50,-2.94
5,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,32,3.896346,-1.280417,92.683067,-40.973333,1.5883,2.2893,2.4833,2.9321,4.9256,4.3516,4.4470,4.6502,4.6768,4.6173,4.5401,4.5133,4.4362,4.2479,4.1439,4.0029,3.9324,3.9037,3.9012,3.8574,3.8483,3.8664,3.8847,3.4724,-0.22,-0.40,-0.58,-0.95,-1.23,-1.23,-1.28,-1.51,-1.76,-1.54,-1.21,-1.27,-1.04,-1.02,-1.19,-1.34,-1.10,-1.28,-1.60,-1.51,-1.60,-1.82,-2.02,-2.03
1,STOCH_RSI_LinearReg,14,35,9,7,3,25,3,0,46,2.991021,-1.283750,91.586958,-59.052500,1.4991,2.0197,2.3874,2.6946,4.2068,3.6713,3.5921,3.6701,3.3410,3.2704,3.2356,3.1836,3.1376,3.0088,2.9427,2.9300,2.8797,2.8375,2.8953,2.8939,2.8948,2.9375,2.9487,2.7063,-0.25,-0.34,-0.67,-1.02,-1.34,-1.26,-1.54,-1.34,-1.40,-1.24,-1.16,-1.06,-1.02,-0.83,-0.79,-0.99,-1.04,-1.29,-1.77,-1.72,-1.89,-2.11,-2.42,-2.32
2,STOCH_RSI_LinearReg,14,35,9,7,3,25,4,0,32,3.727654,-1.612917,87.284933,-51.613333,1.7497,2.4915,2.8835,3.2045,5.1570,4.2845,4.1496,4.2400,4.2091,4.1563,4.0399,4.0584,3.9973,3.8131,3.6952,3.6703,3.6434,3.5946,3.6599,3.6840,3.7027,3.7732,3.7825,3.8235,-0.25,-0.45,-0.82,-1.00,-1.54,-1.98,-2.00,-2.11,-2.07,-1.63,-1.34,-1.36,-1.12,-0.85,-1.06,-0.99,-1.04,-1.59,-2.18,-1.78,-2.25,-2.88,-2.99,-3.43
3,STOCH_RSI_LinearReg,14,35,9,7,3,25,5,0,30,3.816646,-1.505417,84.499375,-45.162500,1.6297,2.3834,2.8014,3.1033,5.1713,4.3351,4.2499,4.4487,4.4221,4.3616,4.2285,4.2107,4.1419,3.9475,3.8163,3.7879,3.7579,3.6978,3.7676,3.7934,3.8134,3.8890,3.8986,3.9425,-0.22,-0.43,-0.75,-0.95,-1.44,-1.17,-1.72,-1.87,-1.81,-1.63,-1.34,-1.36,-1.12,-0.85,-1.06,-0.99,-1.04,-1.45,-1.96,-1.64,-2.20,-2.71,-2.99,-3.43


# Research parameter influence

In [81]:
param = 'LinearReg_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,LinearReg_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,1.937883,-0.562604,65.569898,-39.616042,247
1,3,2.747457,-1.018333,76.114432,-44.385000,123
2,4,4.568197,-2.099375,56.411997,-33.292188,91
3,5,3.548132,-1.371806,74.680826,-39.263194,80
4,6,3.757032,-1.289444,80.305944,-37.298889,80
5,8,3.517467,-1.098056,77.010065,-33.492500,86


# Plot result in file

In [77]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'LinearReg_timeperiod', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

# Save new config data to config file

In [18]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            